In [28]:
import pandas as pd
import re

In [29]:
data=pd.read_csv('data_prepared_time.csv')

In [40]:
data.head()

,Unnamed: 0,Unnamed: 0.1,thread_id,thread_dayofweek,thread_hour,thread_timestamp,text,time_deltas,reply_hour,reply_dayofweek,channel,reply_text,reply_timestamp,users
0,0,0,12781837000,3,9,2017-01-19 09:06:10,<@U1G303UTW> get laid with Bayes,193,9,3,_call_4_collaboration,<@U30Q72KLJ> он же вроде умер,2017-01-19 09:09:23,189
1,1,1,12781879600,3,9,2017-01-19 09:13:16,"Чатег, я вас разочарую - сообщение в дейтинге ...",57,9,3,_call_4_collaboration,<@U2TP5JELS> а как ты проверял?,2017-01-19 09:14:13,189
2,2,2,12782089400,3,9,2017-01-19 09:48:14,"Из `{&lt;женское_имя&gt;; уменьшительное, ласк...",49,9,3,_call_4_collaboration,"`{&lt;женское_имя&gt;; уменьшительное, ласкате...",2017-01-19 09:49:03,189
3,3,3,12844819600,3,16,2017-01-26 16:03:16,Гарри Селдон в восхищении от вашего проекта.,408,16,3,_call_4_collaboration,notabene: ну все-таки надо добавить был бы,2017-01-26 16:10:04,189
4,4,4,12844915400,3,16,2017-01-26 16:19:14,"а можете рассказать, в чем заключается вообще ...",30652,0,4,_call_4_collaboration,"katya: позже напишу, как будет время:)",2017-01-27 00:50:06,189


In [31]:
active=pd.read_csv('active_users1.csv')

In [32]:
active.head()

,channel,users
0,_call_4_collaboration,189
1,_general,176
2,_general_en,11
3,_jobs,573
4,_meetings,422


In [39]:
data = pd.merge(data, active, on="channel", how="left")

In [41]:
#просто перевожу назву каналу в числа, щоб можно було його як фічу використовувати 
channel_names = data['channel'].unique()
channel_names_dictionary = {}

for i, channel in enumerate(channel_names):
    channel_names_dictionary[channel] = i
    
data['channel'] = pd.Series([channel_names_dictionary[channel] for channel in data['channel']], dtype=int)

In [42]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



In [74]:
data.tail()

,Unnamed: 0,Unnamed: 0.1,thread_id,thread_dayofweek,thread_hour,thread_timestamp,text,time_deltas,reply_hour,reply_dayofweek,channel,reply_text,reply_timestamp,users
12556,13024,13024,14182366794,4,11,2017-06-30 11:27:47,Всем привет) Меня зовут Леонид. Я из Хабаровск...,3258,12,4,46,"leoromanovich: привет, земляк! Я из Биробиджан...",2017-06-30 12:22:05,1065
12557,13025,13025,14233624149,3,9,2017-07-06 09:50:41,"Всем привет. Я начинающий в этой области, сейч...",184,9,3,46,valid: привет. Без математики не бывает даже п...,2017-07-06 09:53:45,1065
12558,13026,13026,14234268361,3,11,2017-07-06 11:38:03,Привет всем!\nПо жизни гоняю желтых слонов по ...,61688,4,4,46,apilipenko: и тебя завербовали!,2017-07-07 04:46:11,1065
12559,13027,13027,14306168862,4,19,2017-07-14 19:21:28,"Всем салют! Меня зовут Володя, я из Франкфурта...",61149,12,5,46,<@U698CFT53> Привет из Хайдельберга!,2017-07-15 12:20:37,1065
12560,13028,13028,14329988190,0,13,2017-07-17 13:31:21,"Привет, меня зовут Сергей. Я работаю в Дармшта...",510,13,0,46,&gt;которая кстати активно набирает людей \n\n...,2017-07-17 13:39:51,1065


In [44]:
# можна і прибрати цей макс_фічерс але тоді буде дуже велика матриця (50000 десь)
cv = TfidfVectorizer(
                    analyzer='word', # token = word
                    ngram_range=(1,3), # only unigrams are used, (1,2) - unigrams/bigrams, ..., etc.
                    #stop_words=['my', 'stop', 'word', 'list'], # or stop_words='english'
                    vocabulary=None, # or vocabulary=your_own_dictionary
                    max_df=1.0, # don't filter words by their frequency
                    max_features=1000 # only top-1000 words will be used as columns
                    )

In [45]:
# формуємо матрицю, яка складається з векторів - які відповідають тексту повідомлень
# координати вектора позначають слова
# (розмірність вектора це кількість унікальних слів в нашому датасеті).
transformed_texts_cv = cv.fit_transform(data['text'].values.astype('U'))

In [46]:
print('Obtained feature matrix X:')
A_cv=transformed_texts_cv.todense()
print(transformed_texts_cv.todense(), '\n')

Obtained feature matrix X:
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] 



In [47]:
A_cv.shape

(12561, 1000)

In [48]:
import numpy as np
A_cv_1=np.asarray(A_cv)

In [49]:
(A_cv_1).shape[1]

1000

In [62]:
# ще додатоково обираю 3 фічі - день тиждня і годину коли повідомлення було написано і канал 
data1=data[['thread_dayofweek','thread_hour','channel', 'users']]

In [63]:
A_1 = data1.as_matrix()

In [64]:
print(A_1.shape)
print(A_cv_1.shape)

(12561, 4)
(12561, 1000)


In [65]:
#остаточно отримую свою features matrix 
A_feat=np.concatenate((A_1,A_cv_1),axis=1)

In [66]:
A_feat.shape

(12561, 1004)

In [67]:
# time_deltas це час(в секундах) коли відповіли на повідомлення 
data2=data['time_deltas']
y=data2.as_matrix()
y.shape

(12561,)

In [68]:
print(y)

[  193    57    49 ..., 61688 61149   510]


In [69]:
A_feat.shape

(12561, 1004)

In [70]:
# власне треную свою модель, використовуючи lasso регресію, 
# можна спробувати ділити датасет не рандомом, а ,наприклад, тест брати все після 1 травня чи якось так 
from sklearn import linear_model
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(A_feat, y, test_size=0.3, random_state=42)

#16-18

reg_1 = linear_model.Lasso(alpha = 1,max_iter=1000, tol=0.01)
reg_1.fit(X_train, y_train)

Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.01, warm_start=False)

In [71]:
import numpy as np
#np.sqrt(mse(y_test, reg_1.predict(X_test)))/60

In [72]:
# mae я вже дивлюся в хвилинах - шоб не так печально было)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test/60, (reg_1.predict(X_test))/60)

137.0056287393885